### 5.2.1 FM模型

#### 一. 二阶特征组合
1. 为什么引入二阶特征组合   
 一般做CTR预估的方法, 最后一步是使用逻辑回归预测点击概率.  
 前篇的GBDT+LR的组合, 就是使用GBDT的树型结构, 模拟"苏格拉底式诘问"对特征进行非线性组合的结果.   
 除此之外, 我们想到, 如果任意组合2个特征, 形成二阶特征组合, 则会有更好表现. 此时, 加入二阶特征组合的LR为 : $$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ { w }_{ ij }{ x }_{ i }{ x }_{ j } }  } $$  
2. 二阶特征组合带来的新问题  
  1. 二阶特征组合会使得样本变得更稀疏. 因为二阶特征组合使得特征维度大大增加, 二样本数量还是原来那么多  
  2. 如果样本中, 没有某种二阶组合特征的样本, 则预测新样本是会不准确 (因为没有样本可以在那个参数上进行梯度下降)

#### 二. FM(factorized mechine)原理
1. FM是对超级稀疏的矩阵做参数估计, 并加入二阶特征组合的模型 
2. FM是把二阶特征组合的参数进行2个因子化的积
  1. 对于密集矩阵的参数估计, 一般使用SVM, 但SVM在稀疏矩阵上的参数学习是不可靠的. 这里的FM即便在稀疏矩阵的情况下,学习到的参数也是可靠的   
  2. 对于二阶特征组合的LR, 根据公式$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ { w }_{ ij }{ x }_{ i }{ x }_{ j } }  } $, 发现参数多出了最后面的${ w }_{ ij }$.   
 而上面的分析指出,训练样本可能不存在某些二阶特征组合对应的数据, 此时, 若我们能够找出每个属性$x_i$对应的权重$w_i$的隐因子向量$v_i$, 则2个隐因子向量的点积就能得到对应的二阶特征权重,即${ w }_{ ij }=\left< { v }_{ i },{ v }_{ j } \right> $. 从而有 :  $$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ \left< { v }_{ i },{ v }_{ j } \right> { x }_{ i }{ x }_{ j } }  } $$
 
2. 如果用户u同时对物品i和物品j做出了正向反馈, 则在FM算法中, 物品i和物品j的向量会很相似. 这也符合我们的直觉


3. 二阶特征组合与全职的计算优化  
 因为$\hat { y } =b+{ w }_{ 1 }x+\sum _{ i=1 }^{ n }{ \sum _{ j=i+1 }^{ n }{ \left< { v }_{ i },{ v }_{ j } \right> { x }_{ i }{ x }_{ j } }  } $. 若$v_i$和$v_j$的维度为k, 特征个数为$n$, 则最后一项计算的时间复杂度为$O(kn^2)$. 但FM优化后, 可将这一项的时间复杂度降低为$O(kn)$, 有如下推到
 <img src='img/fmmodel.png' width='85%' height='85%'>
 因为是稀疏矩阵, n个特征大部分为0, 所以时间复杂度从$O(kn)$具体为$O(k{ \bar { m }  }_{ D })$

4. FM做预测
FM模型指出了$\hat { y } $在特征组合下的线性计算方式, 并给出了二阶特征组合相乘的$O(kn)$优化方式. 此外还需知道优化函数, 即可将FM作为预测  
  1. 回归问题 : 优化函数为最小化平方误差   
  2. 二分类问题 : 优化函数为交叉熵  
  3. 排序问题 : 优化函数为BPR的优化函数$\sum _{ \left( u,i,j \right) \in { D }_{ S } }^{  }{ \ln { p\left( i{ > }_{ u }j|\theta  \right)  }  } -{ \lambda  }_{ \theta  }{ \left\| \theta  \right\|  }^{ 2 }$,   
   其中$p\left( i{ > }_{ u }j|\theta  \right)  = sigmoid\left( { x }_{ ui }-{ x }_{ uj } \right) $

5. 梯度下降计算  
 <img src='img/sgdfm2.png' height='90%' width='90%'>

6. FM训练集的构造, 需要把所有特征并列一起, 把是否点击作为标签进行训练
<img src='img/fmtrainset.png' height='70%' width='70%'>

### 5.2.2 FMM模型 - Filed-aware factorization machines

#### 一. FMM模型总览
1. FMM模型有效用于CTR预估(Click-through rate)
2. FFM比FM在各种指标都有提升, 该算法屡次赢得世界范围的CTR预估大赛得奖

#### 二. 回顾FM模型为什么在稀疏矩阵上也很有效的进行CTR
1. 假设数据对$(ESPN, Adidas)$的正例很多, 负例只有一个. 此时传统思路的逻辑回归由于福利样本不足而无法学习到足够合适的权重. 但是FM算法中, 因为$(ESPN, Adidas)$的预测结果, 其二阶参数由$w_{ESPN}*w_{Adidas}$决定, 这两个参数可以由诸如 $(ESPN, Nike)$, $(NBC, Adidas)$的训练数据获得
2. FFM在FM的基础上, 任务属性对应的隐因子向量不应该是1个, 而是$f$个. $f$=属性类别个数.  
 因为做线性回归时, 特征往往需要进行独热编码, 独热编码之前的属性个数, 就是属性的种类$f$. 为了计算出$f$个隐因子向量, 其估计值为

#### 三. FFM的二阶特征组合组合权重计算
$${ \phi  }_{ FFM }\left( w,x \right) =\sum _{ { j }_{ 1 }=1 }^{ n }{ \sum _{ { j }_{ 2 }={ j }_{ 1 }+1 }^{ n }{ \left( { w }_{ { j }_{ 1 }{ f }_{ 2 } }*{ w }_{ { j }_{ 2 }{ f }_{ 1 } } \right) { x }_{ { j }_{ 1 } }{ x }_{ { j }_{ 2 } } }  } $$